# SAS - Распознавание повреждений

## Задача
Обучить модель классификации типов повреждении автомобилей.

## Требуемое качество
Misclassification Error (%) < 30 % на тестовой выборке.

## Файлы и проект
Для создания проекта, Вам необходимо скопировать текущий ноутбук и все файлы из данного репозитория на сервер tiger.cs.msu.ru по SSH в свою домашнюю директорию.
Доступ к проекту может быть получен через JupyterHub (см. ниже).

## Решение
Ноутбук + обученные веса модели + все необходимый дополнительные файлы, если таковые имеются.

## Проверка решения
Ноутбук будет запущен на отдельном сервере с анологичной конфигурацией пакетов и железа.  
Если ноутбук не запускается по причине ошибок в коде, задача отправляется на доработку.  
Результаты модели полученной после запуска ноутбука будут сравниваться с результатами присланной модели.  

## Подключение к CAS
Используйте личную учетную запись.

## Подключение к серверу через SSH
Используйте личную учетную запись.
Для подключения к серверу по SSH рекомендую использовать [MobaXterm](https://mobaxterm.mobatek.net/download.html).

## Jupyter Hub
Ссылка на Jupyter Hub: tiger.cs.msu.ru:8000  
Используйте личную учетную запись.

## Полезные ссылки

[SWAT](https://github.com/sassoftware/python-swat)  
[DLPy](https://github.com/sassoftware/python-dlpy)  
[Deep Learning Programming Guide](https://go.documentation.sas.com/?cdcId=pgmsascdc&cdcVersion=9.4_3.5&docsetId=casdlpg&docsetTarget=titlepage.htm&locale=en)  
[SAS Deep Learning Concepts](https://go.documentation.sas.com/?cdcId=pgmsascdc&cdcVersion=9.4_3.5&docsetId=casdlpg&docsetTarget=n0s7fuxsukkyl6n18qqrblkvn02v.htm&locale=en) 

In [ ]:
import os
import sys
import pandas as pd
from swat import *
import dlpy
from dlpy.utils import *
import dlpy.applications as app
from dlpy.applications import *
from dlpy.model import *
from dlpy.images import *
import dlpy.applications as app
from dlpy.splitting import two_way_split
from dlpy.lr_scheduler import ReduceLROnPlateau, StepLR
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
# os.environ['CAS_CLIENT_SSL_CA_LIST']='/opt/sas/viya/config/etc/SASSecurityCertificateFramework/cacerts/vault-ca.crt'

HOST=''
PORT=5572
USERNAME=''
PASSWORD=''

s = CAS(HOST, PORT, USERNAME, PASSWORD)
s.loadactionset('image')
s.loadactionset('deepLearn')

In [ ]:
HOME_DIR = ''
TRAIN_DIR = os.path.join(HOME_DIR, 'data/prepared_dataset/train/')
TEST_DIR = os.path.join(HOME_DIR, 'data/prepared_dataset/test')
WEIGHTS_DIR = os.path.join(HOME_DIR, 'weights')

# Загрузка данных

In [ ]:
train_tbl = ImageTable.load_files(s, path=TRAIN_DIR, casout=dict(name='train_tbl'))
train_tbl, val_tbl = two_way_split(tbl=train_tbl, test_rate=20, im_table=True, stratify=True)
test_tbl = ImageTable.load_files(s, path=TEST_DIR, casout=dict(name='test_tbl'))

In [ ]:
train_tbl.resize(256,256,inplace=True)
val_tbl.resize(256,256,inplace=True)
test_tbl.resize(256,256,inplace=True)

In [ ]:
train_tbl.label_freq

In [ ]:
val_tbl.label_freq

In [ ]:
train_tbl.show(randomize=True,ncol=3)

In [ ]:
val_tbl.show(randomize=True,ncol=3)

# Построение модели

In [ ]:
from dlpy.sequential import Sequential
from dlpy.layers import InputLayer, Conv2d, BN, Pooling, OutputLayer, Dense, Reshape

In [ ]:
MODEL_TABLE='car_damage_recognition_model'
IN_CHANNELS=3
OUT_CLASSES=8
WIDTH=256
HEIGHT=256
SCALE=1
OFFSETS=(103.939, 116.779, 123.68)

model = Sequential(conn=s, model_table=MODEL_TABLE)

model.add(InputLayer(
    n_channels=IN_CHANNELS,
    width=WIDTH,
    height=HEIGHT,
    name=MODEL_TABLE,
    scale=SCALE,
))

![VGG-11](vgg11.png)

* Размеры всех сверточных фильтров - 3x3  
* Размеры всех пулинг фильтров - 2x2  
* Добавьте после сверточных слоев batch normalization слои

In [ ]:
# TODO: Воссоздайте архитектуру сети VGG-11 в Sequential стиле

In [ ]:
model.print_summary()

In [ ]:
model.plot_network()

# Обучение

In [ ]:
# TODO: Инициализируем step learning rate policy с параметром gamma=0.1

In [ ]:
# TODO: Инициализируем Momentum solver с параметрами  mini_batch_size=4, seed=17, max_epochs=10

## Обучение модели

In [ ]:
model.fit(data=train_tbl, 
           optimizer=optimizer, 
           n_threads=1, 
           seed=17,
           record_seed=17,
           force_equal_padding=True,
           save_best_weights=True,
           valid_table=val_tbl)

In [ ]:
model.plot_training_history(items=['Loss', 'FitError', 'ValidLoss', 'ValidError'])

## Сохранение модели на диск в табличном виде

In [ ]:
model.save_to_table(WEIGHTS_DIR)

# Применение модели

## Загрузка модели с диска в табличном виде

In [ ]:
model = Model(s)
model.load(os.path.join(WEIGHTS_DIR, 'car_damage_recognition_resnet50.sashdat'))

In [ ]:
model.predict(val_tbl,
              layer_out='output',
              layers=['output'],
              mini_batch_buf_size=2,
              use_best_weights=True,
              buffer_size=2,
              n_threads=8,
              log_level=2,
              gpu=gpu)

In [ ]:
model.predict(test_tbl,
              layer_out='output',
              layers=['output'],
              mini_batch_buf_size=2,
              use_best_weights=True,
              buffer_size=2,
              n_threads=8,
              log_level=2,
              gpu=gpu)

# Матрица ошибок

In [ ]:
model.valid_res_tbl.crosstab(row='_label_', col='I__label_')

# Отображение результатов

In [ ]:
model.plot_evaluate_res(img_type='A', n_images=5, randomize=True)